In [66]:
import pandas as pd
import numpy as np
import copy

In [28]:
#Wczytanie danych
dane=pd.read_csv('ml-100k/u.data', sep=' ', header=None)


In [31]:
#Liczba użytkowników
no_users=len(pd.unique(dane[0]))
no_users
#Liczba filmów
#no_movies=len(pd.unique(dane[1]))

#fix bo to nie działało
# import pandas as pd

columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=columns)

# print(ratings.head())


# ratings = ratings.drop('timestamp', axis=1)

# num_users = ratings['user_id'].nunique()
no_movies = ratings['item_id'].nunique()
# print(f"Liczba użytkowników: {num_users}, Liczba filmów: {num_items}")


In [25]:
#sprawdzamy jakie są największe i najmniejsze wartości id użytkownika i filmu
dane[0].max()
dane[0].min()
# dane[1].max()
# dane[1].min()

'1\t1\t5\t874965758'

In [32]:
#wczytałem inaczej nie mam pojęcia dalczego przykłąd z wykłądu nei działa w tym zakresie
data = pd.read_csv('u.data', sep='\t', header=None)

In [33]:

# Sprawdzanie największych i najmniejszych wartości ID użytkownika
user_id_max = data[0].max()
user_id_min = data[0].min()

# Sprawdzanie największych i najmniejszych wartości ID filmu
movie_id_max = data[1].max()
movie_id_min = data[1].min()

print("Największe ID użytkownika:", user_id_max)
print("Najmniejsze ID użytkownika:", user_id_min)
print("Największe ID filmu:", movie_id_max)
print("Najmniejsze ID filmu:", movie_id_min)


Największe ID użytkownika: 943
Najmniejsze ID użytkownika: 1
Największe ID filmu: 1682
Najmniejsze ID filmu: 1


In [34]:
#przerobienie na ndaraay
dane=data.values

In [35]:
#tworzymy tablice ratings

R=np.zeros((no_users,no_movies))

In [36]:
#uzupełniamy tablice ratings wartościami z pliku
for u , m , r, t in dane: #id uzytkownika ,filmu, rating,krok czasowy
    R[u-1,m-1]=r 

In [38]:
#Wczytujemy informacje o filmach ograniczając się do tytułu
movie_info=pd.read_csv('ml-100k/u.item', usecols=(0,1), header=None, sep='|', encoding='latin1')

In [40]:
#tworzymy słownik tłumacząc nr id filmu na jego tytuł
id2name = {}

In [41]:
#Tworzymy słownik tłuamczący index filmu w tablicy R na jego tytuł
row_no2index={}

In [42]:
#Zmieniamy wartość na ndarray
movie_info=movie_info.values

In [43]:
#uszupłniamy słowniki

for idt, t in movie_info:
    id2name[idt]=t
    row_no2index[idt-1]=t

In [46]:
#Funkcja zwracająca infmoracje o podobieństwie użytkowników z tablicy R do danego usera
def similarity(user):
    #lista do przechowywania podobieństw użytkowników
    lista =[]
    #Pętla po indeksach użytkowników
    for i in range (len(R)):
        #Do listy dołączamy podobieństwo cosinusowe i index użytkwonika
        #Skoro używamy prawdopodobieństwa coisinusowego to nie ma probmu z kodowaniem brakujących wartoci
        lista.append([np.dot(user,R[i])/(np.sqrt(np.dot(user,user))*np.sqrt(np.dot(R[i],R[i]))),i])
        #Jeden z elementów odleglosci cosinusowej można policzyć przed pętlą który ?
    return lista

In [54]:
user=R[2]
sum(user>0)
temp=similarity(user)
temp.sort()
temp

C:\Users\msigm\AppData\Local\Temp\ipykernel_18812\1324138137.py:9: RuntimeWarning: invalid value encountered in scalar divide
  lista.append([np.dot(user,R[i])/(np.sqrt(np.dot(user,user))*np.sqrt(np.dot(R[i],R[i]))),i])


[[0.0, 66],
 [0.0, 92],
 [0.0, 96],
 [0.0, 121],
 [0.0, 123],
 [0.0, 131],
 [0.0, 171],
 [0.0, 174],
 [0.0, 236],
 [0.0, 288],
 [0.0, 320],
 [0.0, 468],
 [0.0, 470],
 [0.0, 557],
 [0.0, 564],
 [0.0, 589],
 [0.0, 606],
 [0.0, 764],
 [0.0, 791],
 [0.0, 854],
 [0.0, 887],
 [0.0, 913],
 [0.0, 931],
 [0.00251127503854441, 337],
 [0.0037426025132458207, 224],
 [0.00396832426940583, 207],
 [0.006246199757916196, 917],
 [0.006618771134944745, 669],
 [0.006671881668334048, 730],
 [0.00751603565724005, 689],
 [0.009206922721375197, 910],
 [0.009265517026308595, 638],
 [0.010402754459232461, 777],
 [0.01069530425000912, 311],
 [0.011452271771351354, 186],
 [0.012153208960066027, 762],
 [0.012527341644212295, 36],
 [0.012967485695370587, 584],
 [0.013292356343994007, 217],
 [0.013470533444360345, 476],
 [0.013756857467273544, 365],
 [0.014245354354871629, 397],
 [0.014645694943347045, 603],
 [0.014970410052983283, 336],
 [0.015399657900399568, 768],
 [0.01565356918465416, 590],
 [0.016163464651273

In [51]:
#Funkcja zwracajaca podobieństwa i indexy dla k najabrdzije podobnych użytkowników
def top_k(user , k=15):
    #Obliczamy prawdopodobieństwa dla wszystkich
    temp=similarity(user)
    #Sortujemy tablicę pamiętajac że ejst sortowana rosnąco według pierwszej kolumny
    temp.sort()
    #zwracamy k wartości z konca tablicy bez ostatneigo któy zawsze będzie równy 1
    return temp[-k-1:-1] # aby nie brać samego siebie pod uwagę

In [ ]:
# #alternatywna forma liczenia prawdopodobieństwa- mało istotne ale było na labach chyba
# from sklearn.metrics.pairwise import cosine_similarity
# cosine_similarity(R,R)

# def similarity2(user):
#     lista=[]
#     for i in range(len(R)):
#         lista.append([np.dot(user,R[i])/(np.sqrt(np.dot(user,user))*np.sqrt(np.dot(R[i],R[i]))),i])
#     return lista

# def top_k_2(user,k=15):
#     sim=np.array(similarity2(user))
#     temp=np.argsort(sim)
#     return temp[-k-1:-1], sim[temp[-k:]]

In [56]:
top_k(user)

C:\Users\msigm\AppData\Local\Temp\ipykernel_18812\1324138137.py:9: RuntimeWarning: invalid value encountered in scalar divide
  lista.append([np.dot(user,R[i])/(np.sqrt(np.dot(user,user))*np.sqrt(np.dot(R[i],R[i]))),i])


[[nan, 99984],
 [nan, 99985],
 [nan, 99986],
 [nan, 99987],
 [nan, 99988],
 [nan, 99989],
 [nan, 99990],
 [nan, 99991],
 [nan, 99992],
 [nan, 99993],
 [nan, 99994],
 [nan, 99995],
 [nan, 99996],
 [nan, 99997],
 [nan, 99998]]

In [69]:
#Funkcja wyświetlająca rekomendacje
def recomendation (user, how_many=10, k=15):# user - użytkownik dla którego chcemy rekomendacje, how_many - ile rekomendacji chcemy, k - ile najbardziej podobnych użytkowników bierzemy pod uwagę
    #Wyznaczamy infomracje o k najbardziej podobnych użytkownikach
    most_similar=top_k(user,k)
    #Odczytujemy wartości podobieństw tych użytkowników
    sim=np.array(most_similar)[:,0]
    #i ich indeksy
    indices=np.array(np.array(most_similar)[:,1],dtype=int)
    #Z tablicy R wybieramy wiersze tylko dla tych użytkowników
    sub_R=R[indices]
    
    #Zamineiamy podobieństwa na wagi(to nie jest koneiczne ale ułatwia to interpretację)
    weigths= sim/sum(sim)
    
    #Tworzymy tablicę dla przewidywanych ocen
    pred_ratings=np.zeros(no_movies)
    
    #Dla każdego filmu z osobna
    for m in range(no_movies):
        #Wybieramy rating wskaznych użytkowników dla danego filmu
        ratings=sub_R[:,m]
        #Tworymamy tablicę okreslajaca kto ocenił dany film
        #Ma ona wartości Treu lub False
        who=sub_R[:,m]>0
        if sum(who)>0:
            #Uzupełniamy przewidywny rating dla rozważanego filmu
            #Mnożymy ratignig przez wagi i dzielimy przez sumę wag użytkowników, którzy ocenili film
            #Zastosoway id chroni nas pred dzieleniem przez 0 
            pred_ratings[m]=sum(ratings*weigths)/sum(who*weigths)
            
        #Wszsytkie ratingi są uzupełnione
        #Nie chemy ponownie filmów już ocenionych przez danego usera więc zastępujemy ratingi
        all_ratings=copy.deepcopy(pred_ratings)
        pred_ratings[user>0]=0
        
        #argsoet zwraca indeskys sortowana tablicy
        #Ostatnie indeksy wskazują na najwyższe wartości ratingów
        #Wybieramy tyle indeksów ile określiliśmy parametrem how_many
        #Dodając 1 dostajemy id filmów
        
        for i in np.argsort(pred_ratings)[-how_many:]+1:
            print(id2name[i])
            
        return all_ratings,pred_ratings


In [68]:
#Użytkownik 2
user=R[2]
#Wyśiwetlenie w konsoli rekomendacji dla użytkownika 2
recomendation(user)


C:\Users\msigm\AppData\Local\Temp\ipykernel_18812\1324138137.py:9: RuntimeWarning: invalid value encountered in scalar divide
  lista.append([np.dot(user,R[i])/(np.sqrt(np.dot(user,user))*np.sqrt(np.dot(R[i],R[i]))),i])


Mary Shelley's Frankenstein (1994)
Kid in King Arthur's Court, A (1995)
Interview with the Vampire (1994)
Heavenly Creatures (1994)
Farinelli: il castrato (1994)
Wild Bill (1995)
White Man's Burden (1995)
Waterworld (1995)
Demolition Man (1993)
Scream of Stone (Schrei aus Stein) (1991)


array([0., 0., 0., ..., 0., 0., 0.])

In [80]:
#Definijume funkcje sprawdzającą na ile obliczone ratinig są zgodne z rzeczywistymi taka ocena

def evaluate(user,how_many,k):
    #oceniamy jak dobrez są przydzileane rangi nie same rekomendacje 
    # więc potrzebujemy też wartości ratingów dla danego usera przewidziane przez naszą metodę dodajemy return do fukcnji
    all_ratings,pred_ratings=recomendation(user,how_many,k)
    #poznajemy indeksy ocenionych filmów
    indices=user>0
    true=user[indices]
    pred=all_ratings[indices]
    
    mse=np.power(true-pred,2).sum()/len(true) #różnica jak bardzo się różnią do kwadratu i suma / sume ocenioncyh 
    mae = np.abs(true - pred).sum() / len(true)
    
    print("MSE wyniósł: {:.4f}".format(mse))
    print("MAE wyniósł: {:.4f}".format(mae))
    

#Definujemy funkcję sprawdzajacą skuteczność metody jeśli wrtość obliczonych ratingów za

def evaluate_acc(user,how_many,k):
    #oceniamy jak dobrez są przydzileane rangi nie same rekomendacje 
    # więc potrzebujemy też wartości ratingów dla danego usera przewidziane przez naszą metodę dodajemy return do fukcnji
    all_ratings,pred_ratings=recomendation(user,how_many,k)
    #poznajemy indeksy ocenionych filmów
    indices=user>0
    true=user[indices]
    pred=all_ratings[indices]
    np.round(pred) #zaokrąglamy do najbliższej liczby całkowitej
    
    mse=np.power(true-pred,2).sum()/len(true) #różnica jak bardzo się różnią do kwadratu i suma / sume ocenioncyh 
    mae = np.abs(true - pred).sum() / len(true)
    
    print("MSE wyniósł: {:.4f}".format(mse))
    print("MAE wyniósł: {:.4f}".format(mae))

In [81]:
evaluate(R[1],1,25)
#jest zastosowany trik z wykluczneiem ostatniego użytkownika no niby spoko ale nie wiemy czy uzytkwonik jest w tej bazie więc
#w realnych warunkach lepiej dodać jakiś parametr do top_k na przklad from_trainingset=True

evaluate_acc(R[1],1,25) 

C:\Users\msigm\AppData\Local\Temp\ipykernel_18812\1324138137.py:9: RuntimeWarning: invalid value encountered in scalar divide
  lista.append([np.dot(user,R[i])/(np.sqrt(np.dot(user,user))*np.sqrt(np.dot(R[i],R[i]))),i])


Scream of Stone (Schrei aus Stein) (1991)
MSE wyniósł: 14.8065
MAE wyniósł: 3.7097
Scream of Stone (Schrei aus Stein) (1991)
MSE wyniósł: 14.8065
MAE wyniósł: 3.7097


In [ ]:
#Z modyfikacji to można wstawić zmienną display_recommendations=True żeby je wypisuwało albo nie

In [86]:
#podejścei item based czyli będziem chcieli wybać filmy podobne do wybranego filmu

def similarity_item(movie):

    lista =[]
    movie_sd=np.sqrt(np.dot(movie,movie))
    for i in range (R.shape[1]):
        lista.append([np.dot(movie,R[:,i])/(movie_sd)*(np.sqrt(np.dot(R[:,i],R[:,i]))),i])
    return lista

In [88]:
# Funkcja obliczająca podobieństwo między wybranym filmem a wszystkimi innymi filmami
def similarity_item(movie):
    """
    Oblicza podobieństwo kosinusowe pomiędzy danym filmem a wszystkimi innymi filmami.
    
    Args:
        movie (array): Wektor ocen dla wybranego filmu.
    
    Returns:
        lista (list): Lista zawierająca pary [podobieństwo, indeks filmu] dla wszystkich filmów.
    """
    lista = []  # Lista, która będzie przechowywać wynik
    movie_sd = np.sqrt(np.dot(movie, movie))  # Długość wektora filmu (norma wektora)

    # Iterujemy po wszystkich filmach
    for i in range(R.shape[1]):  # Zakładam, że R to macierz użytkowników x filmów
        # Obliczanie podobieństwa kosinusowego dla filmu o indeksie 'i'
        # (iloczyn skalarny / iloczyn długości wektorów)
        other_movie = R[:, i]
        other_movie_sd = np.sqrt(np.dot(other_movie, other_movie))  # Norma dla innego filmu
        similarity = np.dot(movie, other_movie) / (movie_sd * other_movie_sd) if other_movie_sd > 0 else 0
        
        lista.append([similarity, i])  # Dodanie wyniku do listy

    return lista


In [89]:
movie= R[:,2]
similarity_item(movie)

[[0.3302447874360035, 0],
 [0.2730691753767144, 1],
 [1.0, 2],
 [0.3248663850681052, 3],
 [0.21295655551390094, 4],
 [0.1067222656627104, 5],
 [0.37292068801951556, 6],
 [0.200793891802957, 7],
 [0.2736692755325718, 8],
 [0.15810426409954928, 9],
 [0.36116480017209246, 10],
 [0.3192953773206038, 11],
 [0.2714023989537559, 12],
 [0.1902416934965925, 13],
 [0.26633537379553146, 14],
 [0.16760921791797215, 15],
 [0.40423863309806607, 16],
 [0.12656405075841684, 17],
 [0.1053772827563846, 18],
 [0.18773254111496998, 19],
 [0.19454620357588334, 20],
 [0.2937324642212675, 21],
 [0.23381359005040442, 22],
 [0.38190990249829754, 23],
 [0.306447364690268, 24],
 [0.2141253460057746, 25],
 [0.2613028652016255, 26],
 [0.26929581564895877, 27],
 [0.25307376320359665, 28],
 [0.1263466361680352, 29],
 [0.2591286476599843, 30],
 [0.1851872995902676, 31],
 [0.4049303159741996, 32],
 [0.17900827114065468, 33],
 [0.04631513373928279, 34],
 [0.07333229508719774, 35],
 [0.051269934532826814, 36],
 [0.24053

In [92]:
temp=similarity_item(movie)
np.max(temp,axis=0)
temp.sort()
temp[-10:]

[[0.3972741108792747, 474],
 [0.4016758547313358, 824],
 [0.40408045294158057, 1046],
 [0.40423863309806607, 16],
 [0.4047870212366963, 249],
 [0.4049303159741996, 32],
 [0.42324934635999745, 41],
 [0.44435507440937694, 762],
 [0.45202290913857923, 409],
 [1.0, 2]]

In [93]:
#jesli mamy już obliczone podobne do sibie filmy to czas na rekomandcje

#Funkcja wyświetlająca rekomendacje
def recomendation_item (user, how_many=10, k=15):

    
    pred_ratings=np.zeros(no_movies)
    for m in range(no_movies):
        sim=np.array(similarity_item(R[:,m]))[:,0]
        
        pred_ratings[m]=np.dot(sim,user) / np.dot(sim,user>0)
            
    
    all_ratings=copy.deepcopy(pred_ratings)
    pred_ratings[user>0]=0
        
        
    for i in np.argsort(pred_ratings)[-how_many:]:
        print(id2name[i])
            
        return all_ratings,pred_ratings


In [95]:
import numpy as np
import copy

# Funkcja generująca rekomendacje filmowe w podejściu item-based
def recomendation_item(user, how_many=10, k=15):
    """
    Generuje rekomendacje filmowe dla użytkownika na podstawie podobieństwa kosinusowego między filmami.
    
    Args:
        user (array): Wektor ocen użytkownika (dla każdego filmu).
        how_many (int): Liczba filmów do zarekomendowania (domyślnie 10).
        k (int): Liczba najbardziej podobnych filmów branych pod uwagę (nieużywane w kodzie, ale można rozszerzyć funkcję o tę funkcjonalność).
    
    Returns:
        all_ratings (array): Wszystkie przewidywane oceny dla użytkownika.
        pred_ratings (array): Przewidywane oceny dla filmów, które użytkownik jeszcze nie ocenił.
    """
    
    # Inicjalizacja tablicy na przewidywane oceny dla wszystkich filmów
    pred_ratings = np.zeros(no_movies)

    # Obliczanie przewidywanych ocen dla każdego filmu
    for m in range(no_movies):
        # Obliczenie podobieństw między danym filmem a wszystkimi innymi
        sim = np.array(similarity_item(R[:, m]))[:, 0]
        
        # Przewidywana ocena filmu na podstawie podobieństw i ocen użytkownika
        # Warunek zabezpiecza przed dzieleniem przez zero
        if np.dot(sim, user > 0) > 0:
            pred_ratings[m] = np.dot(sim, user) / np.dot(sim, user > 0)
        else:
            pred_ratings[m] = 0

    # Kopia wszystkich przewidywanych ocen (przed ustawieniem ocenionych filmów na 0)
    all_ratings = copy.deepcopy(pred_ratings)

    # Wyzerowanie przewidywanych ocen dla filmów już ocenionych przez użytkownika
    pred_ratings[user > 0] = 0

    # Sortowanie indeksów filmów według przewidywanych ocen (rosnąco)
    # Następnie wybór najlepszych "how_many" filmów
    top_movie_indices = np.argsort(pred_ratings)[-how_many:]

    # Wyświetlanie rekomendowanych filmów
    print("Rekomendowane filmy:")
    for i in reversed(top_movie_indices):  # Odwrócenie, aby wyświetlać filmy w kolejności malejących ocen
        print(id2name[i])  # id2name zakładamy jako słownik mapujący indeksy na nazwy filmów
    
    return all_ratings, pred_ratings


In [ ]:
user=R[2]
all_r,pred_r=recomendation_item(user,how_many=10, k=15)

In [99]:
#możnap porcedurer liczenia podobieństwa między filmami zrobić jako funkcję

def movie_matrix(R):
    res=np.zeros((R.shape[1],R.shape[1]))
    for m in range(no_movies):
        res[m]=np.array(similarity_item(R[:,m]))[:,0]
    return res



In [ ]:
res=movie_matrix(R)

In [101]:
#u[dae aby korzystala z etej funkcji]
#Funkcja wyświetlająca rekomendacje
def recomendation_item2 (user, how_many=10, k=15):
    pred_ratings=np.zeros(no_movies)
    for m in range(no_movies):
        #sim=np.array(similarity_item(R[:,m]))[:,0]
        
        pred_ratings[m]=np.dot(res[m],user) / np.dot(res[m],user>0)
            
    
    all_ratings=copy.deepcopy(pred_ratings)
    pred_ratings[user>0]=0
        
        
    for i in np.argsort(pred_ratings)[-how_many:]:
        print(id2name[i])
            
        return all_ratings,pred_ratings


In [ ]:
all_r,pred_r=recomendation_item2(user,how_many=10, k=15)